# Starting point: end of day 1

In [ ]:
import logging
import re

def entries_counter(source_file):
    matches = re.findall('^$', open(source_file, 'r').read(), flags=re.MULTILINE)
    nb_blocks = len(matches)
    logging.debug(f'{nb_blocks} blocks in the file "{source_file}".')
    return nb_blocks

def file_splitter(source_file_name, number_of_files, separator='\n', output_name='split'):
    logging.info(f'Start to split "{source_file_name}" in {number_of_files} files.')
    total_blocs_in_file = entries_counter(source_file=source_file_name)
    max_blocs_in_file = round(total_blocs_in_file/float(number_of_files), 0)
    logging.debug(f'{max_blocs_in_file} blocks per file.')
    
    padding_length = len(str(number_of_files))
    
    with open(source_file_name, 'r') as original_file:
        file_number = 1
        blocs_in_file = 0
        new_file_content = ''

        # Loop through the file, line by line
        for line in original_file:
            # Store the line in a temporary variable
            new_file_content += line
            if line == separator:
                # Count the blocks
                blocs_in_file += 1
            if blocs_in_file > max_blocs_in_file:
                logging.debug(f'Writing {output_name}_{file_number:0{padding_length}}.txt.')
                # If we reach the limit, write the content of the temporary variable in the new file
                with open(f'{output_name}_{file_number:0{padding_length}}.txt', 'w') as new_file:
                    new_file.write(new_file_content)
                # Reset counters
                file_number += 1
                blocs_in_file = 0
                new_file_content = ''
        else:
            logging.debug(f'Writing {output_name}_{file_number:0{padding_length}}.txt.')
            # EOF reached, writing everything we have in the temporary variable
            with open(f'{output_name}_{file_number:0{padding_length}}.txt', 'w') as new_file:
                new_file.write(new_file_content)
    logging.info(f'Done with "{source_file_name}".')

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

file_splitter(source_file_name='data/bview.20030809.1600.txt', number_of_files=100, separator='\n', output_name='split')

# Validator

In [ ]:
import glob
import hashlib

with open('data/bview.20030809.1600.txt', 'rb') as f:
    hash_source = hashlib.sha256(f.read()).hexdigest()
print(hash_source)

hash_dest = hashlib.sha256()
for out_file in sorted(glob.glob('split_*.txt')):
    with open(out_file, 'rb') as f:
        hash_dest.update(f.read())
dest = hash_dest.hexdigest()
print(dest)

# Profiling

Install dependencies

In [ ]:
%%bash

pip install line-profiler memory_profiler

In [ ]:
%load_ext memory_profiler
%load_ext line_profiler

## CPU

In [ ]:
%time file_splitter(source_file_name='data/bview.20030809.1600.txt', number_of_files=10, separator='\n', output_name='split')

In [ ]:
%%timeit

file_splitter(source_file_name='data/bview.20030809.1600.txt', number_of_files=10, separator='\n', output_name='split')

In [ ]:
%prun file_splitter(source_file_name='data/bview.20030809.1600.txt', number_of_files=10, separator='\n', output_name='split')

In [ ]:
%prun file_splitter(source_file_name='data/bview.20180216.0800.txt', number_of_files=10, separator='\n', output_name='split')

## Memory

In [ ]:
%%memit 

file_splitter(source_file_name='data/bview.20030809.1600.txt', number_of_files=10, separator='\n', output_name='split')


## Step 1 - Complete rewrite! 

Let's think a bit how we can make this code more efficient. For that, let's think a bit more about our goals: writing N files of more or less the same length.

* How can we do that without counting the amount of blocs?
* Is there a way to move in a file without reading every line?

### TODO:

* Get the size of the file: https://docs.python.org/3/library/os.path.html#os.path.getsize)
* Open the file as bytes: https://docs.python.org/3/library/functions.html#open
* Use `seek` to move in the file, `tell` to figure out where you are, and `read` to copy file chunks (https://docs.python.org/3/tutorial/inputoutput.html?highlight=tell%20file#methods-of-file-objects)

## Step 2 - Pythonize the rewrite


Let's make it even better.

### TODO

* Only open the source file once
* Use `pathlib.Path`: https://docs.python.org/3/library/pathlib.html#concrete-paths
* Get the size of the file with `stat`: https://docs.python.org/3/library/pathlib.html#pathlib.Path.stat

## Step 3 - Make it a class

## Step 4 - Write test cases

## Step 5 - Python Module

## Step 6 - More cool tweaks

* Yield pseudo files (`BytesIO`) instead of writing the files on the disk
* Use `argparse` to make the script easier to use

## Step 7 ++ - Integrate the module in something bigger


If you're fast and bored:

* Fetch new files when there is something available
    * http://data.ris.ripe.net/rrc00/latest-bview.gz
    * ===> http://docs.python-requests.org/en/master/api/#requests.head & Last-Modified

* Use the library to generate text files:
    * https://bitbucket.org/ripencc/bgpdump/downloads/ (Installation details: https://bitbucket.org/ripencc/bgpdump/wiki/Home.wiki#!building)

    ```
    sh ./bootstrap.sh
    make
    ./bgpdump -T
    ```

    ./bgpdump -O ../data/latest-bview.txt  ../data/original/latest-bview.gz